Le but de ce code est de modifier comme il se doit le data set ytrain pour avoir des données par heures et dans un carré


In [2]:
import folium
import pandas as pd
from folium.plugins import TimestampedGeoJson
from shapely.geometry import Point
import numpy as np

from branca.colormap import linear

In [ ]:
data = pd.read_csv("hourly_df_semaine.csv")
df_real = pd.DataFrame(data)
data_stat = pd.read_csv("info_static.csv")
df_stat = pd.DataFrame(data_stat)
df_stat = df_stat[['t_id','latitude','longitude']]

In [8]:

# Nettoyage des données de latitude et longitude dans data_stat
df_stat['latitude'] = pd.to_numeric(df_stat['latitude'], errors='coerce')
df_stat['longitude'] = pd.to_numeric(df_stat['longitude'], errors='coerce')

# Définir les limites de la grille (basé sur Paris)
lat_min, lat_max = 48.81, 48.92
lon_min, lon_max = 2.255, 2.42
step = 0.005  # Taille d'un carré (grille de 0.005°)

In [9]:

# Fonction pour créer la grille
def creer_grille(lat_min, lat_max, lon_min, lon_max, step):
    grille = []
    latitudes = np.arange(lat_min, lat_max, step)
    longitudes = np.arange(lon_min, lon_max, step)
    
    for i in range(len(latitudes) - 1):
        for j in range(len(longitudes) - 1):
            carre = {
                "id": f"{i}_{j}",
                "lat_min": latitudes[i],
                "lat_max": latitudes[i + 1],
                "lon_min": longitudes[j],
                "lon_max": longitudes[j + 1],
                "lati": latitudes[i] + step / 2,  # Centre du carré
                "long": longitudes[j] + step / 2  # Centre du carré
            }
            grille.append(carre)
    return grille

# Fonction pour trouver le carré correspondant à un point
def trouver_carre(lat, lon, grille):
    if pd.isnull(lat) or pd.isnull(lon):  # Vérifier si lat ou lon est NaN
        return None, None, None
    for carre in grille:
        if carre["lat_min"] <= lat < carre["lat_max"] and carre["lon_min"] <= lon < carre["lon_max"]:
            return carre["id"], carre["lati"], carre["long"]
    return None, None, None

# Fonction pour associer les carrés à chaque point GPS
def associer_carres(df, grille):
    df[['carre_id', 'lati', 'long']] = df.apply(
        lambda row: pd.Series(trouver_carre(row['latitude'], row['longitude'], grille)), axis=1
    )
    return df

In [10]:


# Créer la grille
grille = creer_grille(lat_min, lat_max, lon_min, lon_max, step)

# Associer les carrés pour le dataset statique (info_static)
df_stat = associer_carres(df_stat, grille)

# Extraire les identifiants uniques pour correspondre les datasets
df_real['timestamp'] = pd.to_datetime(df_real['timestamp'])  # Convertir en datetime
df_real_long = df_real.melt(id_vars=['timestamp'], var_name='t_id', value_name='status')

# Ajouter les informations statiques (latitude, longitude, adresse, etc.) à partir de data_stat
df_real_long = df_real_long.merge(
    df_stat[['t_id', 'latitude', 'longitude']],
    on='t_id', how='left'
)

# Ajouter les informations de la grille au dataset final
df_real_long = associer_carres(df_real_long, grille)

# Supprimer les lignes avec des valeurs manquantes critiques
df_real_long = df_real_long.dropna(subset=['latitude', 'longitude', 'carre_id'])

# Afficher un aperçu du DataFrame final
print(df_real_long.head(2))



Empty DataFrame
Columns: [timestamp, t_id, status, latitude, longitude, carre_id, lati, long]
Index: []


In [ ]:
# Regrouper les données par carré et t_1h, puis calculer la moyenne de k
resultats_groupes = merged_df.groupby(['timestamp','carre_id']).agg(
    moyenne_k=('k', 'mean'),
    somme_k=('k', 'sum'),
    count_k=('k', 'count'),
    lati= ('lati', 'mean'),
    long= ('long', 'mean')
).reset_index()

print(resultats_groupes['moyenne_k'].max())
resultats_groupes.head(10)
print(resultats_groupes['moyenne_k'].describe())

In [ ]:
# Sauvegarder le DataFrame final dans un nouveau fichier CSV
df_real_long.to_csv("dataset_complet.csv", index=False)
